In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/watermarking

/content/drive/MyDrive/watermarking


In [ ]:
from skimage import io,img_as_float
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import tensorflow as tf
import pandas as pd
import scipy

In [ ]:
path='./Images'
dataset=[]
for img in os.scandir(path):
    dataset.append((img.name,img.path))

In [ ]:
import numpy as np
import pywt
import os
from PIL import Image
from scipy.fftpack import dct
from scipy.fftpack import idct

watermark = 'mark.png' 

In [ ]:


def convert_image(img_path, size,channel=1):
    img = Image.open(img_path).resize((size, size), 1)
    if channel==1:
      img = img.convert('L')
    image_array = np.array(img.getdata(), dtype=np.float).reshape((size, size,channel))
    #print(image_array[0][0])           
    #print(image_array[10][10])           
    return image_array

In [ ]:
def process_coefficients(imArray, model, level):
    coeffs=pywt.wavedec2(data = imArray, wavelet = model, level = level)
    # print coeffs[0].__len__()
    coeffs_H=list(coeffs) 
   
    return coeffs_H


In [ ]:

def embed_mod2(coeff_image, coeff_watermark, offset=0):
    for i in xrange(coeff_watermark.__len__()):
        for j in xrange(coeff_watermark[i].__len__()):
            coeff_image[i*2+offset][j*2+offset] = coeff_watermark[i][j]

    return coeff_image

In [ ]:
def embed_mod4(coeff_image, coeff_watermark):
    for i in xrange(coeff_watermark.__len__()):
        for j in xrange(coeff_watermark[i].__len__()):
            coeff_image[i*4][j*4] = coeff_watermark[i][j]

    return coeff_image

In [ ]:
def embed_watermark(watermark_array, orig_image):
    watermark_array_size = watermark_array[0].__len__()
    watermark_flat = watermark_array.ravel()
    ind = 0

    for x in range (0, orig_image.__len__(), 8):
        for y in range (0, orig_image.__len__(), 8):
            if ind < watermark_flat.__len__():
                subdct = orig_image[x:x+8, y:y+8]
                subdct[5][5] = watermark_flat[ind]
                orig_image[x:x+8, y:y+8] = subdct
                ind += 1 


    return orig_image

In [ ]:

def apply_dct(image_array):
    size = image_array[0].__len__()
    all_subdct = np.empty((size, size))
    for i in range (0, size, 8):
        for j in range (0, size, 8):
            subpixels = image_array[i:i+8, j:j+8]
            subdct = dct(dct(subpixels.T, norm="ortho").T, norm="ortho")
            all_subdct[i:i+8, j:j+8] = subdct

    return all_subdct


In [ ]:

def inverse_dct(all_subdct):
    size = all_subdct[0].__len__()
    all_subidct = np.empty((size, size))
    for i in range (0, size, 8):
        for j in range (0, size, 8):
            subidct = idct(idct(all_subdct[i:i+8, j:j+8].T, norm="ortho").T, norm="ortho")
            all_subidct[i:i+8, j:j+8] = subidct

    return all_subidct

In [ ]:
def get_watermark(dct_watermarked_coeff, watermark_size):
    
    subwatermarks = []

    for x in range (0, dct_watermarked_coeff.__len__(), 8):
        for y in range (0, dct_watermarked_coeff.__len__(), 8):
            coeff_slice = dct_watermarked_coeff[x:x+8, y:y+8]
            subwatermarks.append(coeff_slice[5][5])

    watermark = np.array(subwatermarks).reshape(watermark_size, watermark_size)

    return watermark

In [ ]:
def recover_watermark(image_array, model='haar', level = 1, img_name=""):


    coeffs_watermarked_image = process_coefficients(image_array, model, level=level)
    dct_watermarked_coeff = apply_dct(coeffs_watermarked_image[0])
    
    watermark_array = get_watermark(dct_watermarked_coeff, 128)

    watermark_array =  np.uint8(watermark_array)

#Save result
    img = Image.fromarray(watermark_array)
    img.save('./ExtractedImages/'+img_name)

In [ ]:
def print_image_from_array(image_array, name):
  
    image_array_copy = image_array.clip(0, 255)
    image_array_copy = image_array_copy.astype("uint8")
    img = Image.fromarray(image_array_copy)
    img.save('./output/' + name)

In [ ]:
def w2d(image):
    model = 'haar'
    level = 1
    image_array = convert_image(image[1], 2048,channel=3)
    watermark_array = convert_image(watermark, 128)

    coeffs_image = process_coefficients(image_array[:,:,0], model, level=level)
    dct_array = apply_dct(coeffs_image[0])
    dct_array = embed_watermark(watermark_array, dct_array)
    coeffs_image[0] = inverse_dct(dct_array)
  

# reconstruction
    image_array_H=pywt.waverec2(coeffs_image, model)
    image_array[:,:,0]=image_array_H
    print_image_from_array(image_array, image[0])



# recover images
    recover_watermark(image_array = image_array_H, model=model, level = level,img_name=image[0])


In [ ]:
import math
import numpy as np

def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))
	


In [ ]:
import math
import numpy as np
import cv2

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')
		

In [ ]:
for img in dataset:
  w2d(img)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys


[54. 76. 97.]
[14. 25. 34.]
[255.]
[255.]
[120.  89.  59.]
[115.  80.  53.]
[255.]
[255.]
[237. 207. 188.]
[237. 206. 185.]
[255.]
[255.]
[84. 80. 56.]
[100. 100. 112.]
[255.]
[255.]
[39. 37. 44.]
[44. 42. 53.]
[255.]
[255.]
[234. 233. 233.]
[235. 235. 235.]
[255.]
[255.]
[226. 239. 248.]
[226. 239. 248.]
[255.]
[255.]
[ 56. 127. 155.]
[ 56. 131. 160.]
[255.]
[255.]
[116.  50.  23.]
[176. 117. 142.]
[255.]
[255.]
[150. 180. 224.]
[152. 181. 225.]
[255.]
[255.]
[  4.  56. 128.]
[  3.  56. 129.]
[255.]
[255.]
[182.  62.  74.]
[176.  55.  65.]
[255.]
[255.]
[175. 168. 132.]
[173. 162. 131.]
[255.]
[255.]
[165. 207. 249.]
[165. 207. 250.]
[255.]
[255.]
[163. 155. 150.]
[153. 151. 151.]
[255.]
[255.]


In [ ]:
size = 2048
print('Image','\t\t\t','PSNR','\t\t\t','SSMI')
for img in dataset:
  o_image = './output/'+img[0]
  src = np.array(Image.open(o_image).resize((size, size), 1))
  i_image = img[1]
  des = np.array(Image.open(i_image).resize((size, size), 1))
  psnr = calculate_psnr(src,des)
  ssmi = calculate_ssim(src,des)
  print(img[0],'\t',psnr,'\t',ssmi)

Image 			 PSNR 			 SSMI
00007_TE_1472x976.png 	 31.003304567864824 	 0.8833856836935611
00011_TE_1744x1160.png 	 30.907708168891006 	 0.9084322823510109
00010_TE_2000x1128.png 	 30.935113027913456 	 0.8610820191969663
00009_TE_1976x1312.png 	 31.041740398907457 	 0.9270979120974236
00004_TE_1808x1352.png 	 30.870326548357422 	 0.8976787094591029
00008_TE_1912x1272.png 	 31.089445431413818 	 0.8744052811719928
00003_TE_1944x1296.png 	 31.17271431760976 	 0.865963493480829
00001_TE_2096x1400.png 	 31.314902876970073 	 0.8892013697996527
00013_TE_1920x1280.png 	 30.925377297601187 	 0.8777809945993013
00006_TE_1544x1120.png 	 30.959790941838623 	 0.87458990032375
00005_TE_1336x872.png 	 31.25138298371425 	 0.8490067171494199
00012_TE_1512x2016.png 	 30.990565654828814 	 0.8842616261680248
00002_TE_2144x1424.png 	 30.947140568178092 	 0.8783449055527738
00014_TE_3680x2456.png 	 30.942116335112782 	 0.8661662995588381
